In [1]:
import qiskit
qiskit.__version__

'0.25.0.dev0+c29887a'

# Why singletons?

In [2]:
from qiskit.circuit.library import XGate
gate = XGate()
print(f"name: {gate.name}\n")
print(f"params: {gate.params}\n")
print(f"matrix:\n{gate.to_matrix()}\n")
print(f"definition:\n{gate.definition}\n")

name: x

params: []

matrix:
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]

definition:
   ┌───────────┐
q: ┤ U3(π,0,π) ├
   └───────────┘



In [3]:
new_gate = XGate()
print(f"name: {gate.name}\n")
print(f"params: {gate.params}\n")
print(f"matrix:\n{gate.to_matrix()}\n")
print(f"definition:\n{gate.definition}\n")

name: x

params: []

matrix:
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]

definition:
   ┌───────────┐
q: ┤ U3(π,0,π) ├
   └───────────┘



Previously, every time you ran `XGate()` (or `QuantumCircuit.x(...)`) it creates a new object in memory to represent the same exact thing. This adds up as the number of instructions in the circuit grows.

# Implementation in [#10134](https://github.com/Qiskit/qiskit-terra/pull/10314)

This PR introduces a new class `SingletonGate` which enables using a global shared instance for repeated instances of the class. The parent class for standard library gates that don't take a parameter and aren't a subclass of `ControlledGate`. This includes:


* `DCXGate`
* `ECRGate`
* `HGate`
* `IGate`
* `iSwapGate`
* `SGate`
* `SdgGate`
* `SwapGate`
* `SXGate`
* `SXdgGate`
* `TGate`
* `TdgGate`
* `XGate`
* `RCCXGate`
* `RC3XGate`
* `YGate`
* `ZGate`

In [4]:
XGate() is XGate() is XGate() is XGate() is XGate() is XGate() is XGate() is XGate() is XGate() is XGate()

True

However the Qiskit circuit data model allows for certain fields to customize an instance. This can't be used with a shared single object.

In [5]:
try:
    XGate().label = "My Extra Special XGate made with secret sauce"
except NotImplementedError as e:
    print(e)

This gate class <class 'qiskit.circuit.library.standard_gates.x.XGate'> does not support manually setting a label on an instance. Instead you must set the label when instantiating a new object.


In [6]:
labelled_gate = XGate(label="My Extra Special XGate made with secret sauce")
print(labelled_gate.label)
print(f"Shared object: {labelled_gate is XGate()}")

My Extra Special XGate made with secret sauce
Shared object: False


In [7]:
gate = XGate()
gate.mutable

False

In [8]:
mutable_gate = gate.to_mutable()
mutable_gate.label = "My Extra Special XGate made with secret sauce"
print(mutable_gate.label)
print(f"Shared object: {mutable_gate is XGate()}")

My Extra Special XGate made with secret sauce
Shared object: False


While only label is shown here this also applies to `.condition`, `.unit`, and `.duration` which are the other optional state that can be attached to a specific instance of a gate object. The one special case is `.c_if` which can still work as before to apply a condition to an existing gate, it always returned a new object and this continues to work as before.

In [9]:
from qiskit.circuit import Clbit
gate = XGate()
conditional_gate = gate.c_if(Clbit(), 0)
print(conditional_gate.condition)
print(f"Shared object: {conditional_gate is XGate()}")

(<qiskit.circuit.classicalregister.Clbit object at 0x7f1f576f2080>, False)
Shared object: False


In [10]:
from qiskit.circuit import QuantumCircuit
qc = QuantumCircuit(1, 1)
qc.x(0).c_if(qc.clbits[0], 1)
print(qc)

        ┌───┐   
  q: ───┤ X ├───
        └─╥─┘   
     ┌────╨────┐
c: 1/╡ c_0=0x1 ╞
     └─────────┘


In [11]:
print(qc.data[0].operation is XGate())

False


[#10314](https://github.com/Qiskit/qiskit-terra/pull/10314) uses the new `SingletonGate` class as the parent for all the standard library gates in Qiskit which do not take a parameter and are not a subclass of `ControlledGate`.

The next steps will be to expand this for standard library `ControlledGate` subclasses as there are 2 concrete instances for each of those (based on the control state). Following on from that to enable singletons for gates that take parameters is more complicated as the parameter value makes each instance unique. The path I'm considering is to store a unique instance for each parameter value, in other words there is a shared instance for each time `RZGate(pi/2)` is instantiated, and if you then run `RZGate(pi)` that will be a separate global shared instance. There is longer term work looking at changing the data model to faciliate more memory efficient usage of parameterized gates.

# Performance improvements

For repeated instances of any `SingletonGate` the memory overhead decreased from ~500 bytes/per gate to a single reference which is 8 bytes. Also the runtime overhead for `copy()`/`deepcopy()` is signficantly improved because a copy is just a reference now. For our nightly benchmarks it is showing > 3x faster for some copy benchmarks.